In [81]:
import numpy as np
import pandas as pd

In [82]:
df= pd.read_csv("/content/clean_car_data.csv")

In [83]:
df.head(2)

,Unnamed: 0,name,company,year,Price,kms_driven,fuel_type
0,0,Hyundai Santro Xing,Hyundai,2007,80000,45000,Petrol
1,1,Mahindra Jeep CL550,Mahindra,2006,425000,40,Diesel


In [84]:
df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [85]:
df['company'].nunique()

25

In [86]:
df.nunique()

,0
name,254
company,25
year,21
Price,271
kms_driven,246
fuel_type,3


In [87]:
df.sample(5)

,name,company,year,Price,kms_driven,fuel_type
264,Renault Duster 85,Renault,2015,549999,19000,Diesel
692,Maruti Suzuki Swift,Maruti,2014,260000,120000,Diesel
145,Renault Kwid,Renault,2016,244999,11000,Petrol
543,Chevrolet Enjoy,Chevrolet,2014,490000,30201,Diesel
474,Hyundai Santro Xing,Hyundai,2003,120000,50000,Petrol


In [88]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

In [89]:
ohe= OneHotEncoder()


In [90]:
 x=df[['name', 'company', 'fuel_type', 'kms_driven', 'year']]

In [91]:
y=df['Price']

In [92]:
ohe.fit(x[['name','company','fuel_type']])

OneHotEncoder()

In [93]:
column_trans=make_column_transformer((OneHotEncoder(handle_unknown='ignore'),['name','company','fuel_type']),
                                    remainder='passthrough')

In [94]:
column_trans.feature_names_in_

AttributeError: 'ColumnTransformer' object has no attribute 'feature_names_in_'

In [95]:
from sklearn.model_selection import train_test_split

In [96]:
x_train,x_test,y_train,y_test= train_test_split(x,y,test_size=0.3,random_state=42)

In [97]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline

In [98]:
lr= LinearRegression()

In [99]:
from sklearn.metrics import accuracy_score,r2_score

In [100]:
pipe = make_pipeline(column_trans,lr)

In [101]:
pipe.fit(x_train,y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['name', 'company',
                                                   'fuel_type'])])),
                ('linearregression', LinearRegression())])

In [102]:
y_pred3=pipe.predict(x_test)

In [103]:
r2_score(y_test,y_pred3)

0.6126464643937978

In [104]:
scores=[]
for i in range(1000):
    x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1,random_state=i)
    lr=LinearRegression()
    pipe=make_pipeline(column_trans,lr)
    pipe.fit(x_train,y_train)
    y_pred=pipe.predict(x_test)
    scores.append(r2_score(y_test,y_pred))

In [105]:
np.argmax(scores)

np.int64(302)

In [106]:
scores[np.argmax(scores)]

0.899110882739444

In [107]:
import pickle

In [108]:
pickle.dump(ohe,open('final_encoders.pkl','wb'))

In [109]:
pickle.dump(pipe,open('predictor_model.pkl','wb'))

In [110]:
pipe.predict(pd.DataFrame(columns=['name','company','year','kms_driven','fuel_type'],data=np.array(['Maruti Suzuki Swift','Maruti',2019,100,'Petrol']).reshape(1,5)))

array([430197.70614411])